In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
import tensorflowjs as tfjs

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import os, cv2, glob, json, base64

# from PIL import Image
from blend_modes import normal


# Lets make sure working directory is project root
if os.getcwd().rsplit(os.path.sep,1)[1]=="model_training":
    os.chdir( os.getcwd().rsplit(os.path.sep,1)[0] )

# Load up our lookup_table used for labels
with open(os.path.join("public","lookup_table.json")) as json_file:
    lookup_table  = json.load(json_file)

# Define functions used throughout the document
def read_png(path):
    image = cv2.imread(path,-1)
    return cv2.cvtColor(image,cv2.COLOR_BGR2BGRA) if len(image.shape) > 2 and image.shape[2] == 3 else image

def read_jpeg(path):
    BGR = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    BGRA = cv2.cvtColor(BGR,cv2.COLOR_BGR2BGRA)
    BGRA[:,:,3] = 255
    return BGRA
    
def layer( image_a, image_b ):
    return normal(image_a.astype(float), image_b.astype(float), 1.).astype(np.uint8)

In [ ]:
def generate_champion_and_skin_training_images( champion_image_path ):
    
    # 1.A loading and preparing assets
    champion = cv2.copyMakeBorder(read_jpeg(champion_image_path), 0, 5, 5, 0, cv2.BORDER_CONSTANT,value=(0,0,0,255))
    shard_border = cv2.resize(read_png(os.path.join("model_training","assets","border_images","shard.png")), (385,385))
    permanent_border = cv2.resize(read_png(os.path.join("model_training","assets","border_images","permanent.png")), (385,385))
    token_6_border = read_png(os.path.join("model_training","assets","border_images","champion_token_6.png"))[5:74, :]
    token_6_border = cv2.resize(token_6_border, (385,385))
    token_7_border = read_png(os.path.join("model_training","assets","border_images","champion_token_7.png"))[5:74, :]
    token_7_border = cv2.resize(token_7_border, (385,385))
    quantity_background = cv2.resize(read_png(os.path.join("model_training","assets","border_images","quantity_background.png")), (385,385))

    champion_id = champion_image_path.rsplit(os.path.sep,1)[1].split('.')[0]
    price = lookup_table[champion_id][1]

    if lookup_table[champion_id][2] == 1: legacy_icon = read_png(os.path.join("model_training","assets","tag_icons","legacy.png"))
        
    if price == "special": rarity_icon = read_png(os.path.join("model_training","assets","rarity_icons","rarity_mythic.png"))
    if price == 3250: rarity_icon = read_png(os.path.join("model_training","assets","rarity_icons","rarity_ultimate.png"))
    if price == 1820: rarity_icon = read_png(os.path.join("model_training","assets","rarity_icons","rarity_legendary.png"))
    if price == 1350: rarity_icon = read_png(os.path.join("model_training","assets","rarity_icons","rarity_epic.png"))

    # 1.B Resize rarity and legacy icons to 395x395 pixel resolution overlay images
    if 'legacy_icon' in locals():
        legacy_icon = cv2.resize(legacy_icon, (110,110))
        legacy_icon = cv2.copyMakeBorder(legacy_icon, 291, 0, 0, 297, cv2.BORDER_CONSTANT,value=(0,0,0,0))
        legacy_icon = legacy_icon[0:395,legacy_icon.shape[1]-395:legacy_icon.shape[1]]

    if 'rarity_icon' in locals():
        rarity_icon = cv2.resize(rarity_icon, (385,int(385/rarity_icon.shape[1]*rarity_icon.shape[0])))
        rarity_icon = cv2.copyMakeBorder(rarity_icon, 285, 5, 5, 5, cv2.BORDER_CONSTANT,value=(0,0,0,0))
        rarity_icon = rarity_icon[0:395,0:395]

    # 2. Set up borders this asset uses in the loot tab
    if champion_id.endswith("000"):
        borders = (shard_border, permanent_border, token_6_border, token_7_border)
    else:
        borders = (shard_border, permanent_border)
    
    # 3. Create variations of assets from layers
    for border_index in range(len(borders)):
        for shadow in (False, True):
            image = layer(champion, quantity_background) if shadow else champion
            image = layer(image, borders[border_index])
            image = cv2.copyMakeBorder(image, 5, 5, 5, 5, cv2.BORDER_CONSTANT,value=(14,14,14,255))            

            y_range_increment = 2
            if not champion_id.endswith("000"):
                y_range_increment = 1
                if 'legacy_icon' in locals(): image = layer(image, legacy_icon)  
                if 'rarity_icon' in locals(): image = layer(image, rarity_icon)  
                    
            image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)

          
            # Create different crops to accomodate the imperfection of client screenshot percision
            for x in range(0, 13, 2):
                for y in range(1, 13, y_range_increment):
                    crop = image[0+y:image.shape[1]-10+y, 0+x:image.shape[1]-10+x]              
                    resized_image = cv2.resize(crop, (28,28), interpolation = cv2.INTER_AREA)
                    
                    # plt.figure()
                    # plt.imshow(resized_image)
                    # plt.colorbar()
                    # plt.show()

                    destination = os.path.join("model_training","training_data",champion_id,f'{("shard","permanent","token6","token7")[border_index]}_{"yes" if shadow else "no"}-shadow_x{x}_y{y}.png')
                    if not os.path.exists(destination.rsplit(os.path.sep,1)[0]):
                        os.makedirs(destination.rsplit(os.path.sep,1)[0])

                    cv2.imwrite(destination, resized_image)

In [ ]:
def generate_ward_skin_training_images( ward_image_path ):

    # 1.A loading and preparing assets
    ward = read_png(ward_image_path)
    ward = cv2.copyMakeBorder(ward[0:ward.shape[1],:], 0, 7, 7, 0, cv2.BORDER_CONSTANT,value=(0,0,0,0))
    ward_background = cv2.resize(read_png(os.path.join("model_training","assets","border_images","wardskin_background.png")), (467,467))
    shard_border = cv2.resize(read_png(os.path.join("model_training","assets","border_images","shard.png")), (467,467))
    permanent_border = cv2.resize(read_png(os.path.join("model_training","assets","border_images","permanent.png")), (467,467))
    quantity_background = cv2.resize(read_png(os.path.join("model_training","assets","border_images","quantity_background.png")), (467,467))

    ward_id = "w" + ward_image_path.rsplit("_",1)[1].split(".")[0]

    borders = (shard_border, permanent_border)
        
    # 2. Create variations of assets from layers
    for border_index in range(len(borders)):
        for shadow in (False, True):
            image = layer(ward_background, quantity_background) if shadow else ward_background
            image = layer(image, ward)
            image = layer(image, borders[border_index])
            image = cv2.copyMakeBorder(image, 5, 5, 5, 5, cv2.BORDER_CONSTANT,value=(14,14,14,255))
                    
            image = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)
        
            # Create different crops to accomodate the imperfection of client screenshot percision
            for x in range(0, 13, 2):
                for y in range(1, 13, 1):
                    crop = image[0+y:image.shape[1]-10+y, 0+x:image.shape[1]-10+x]              
                    resized_image = cv2.resize(crop, (28,28), interpolation = cv2.INTER_AREA)

                    destination = os.path.join("model_training","training_data",ward_id,f'{("shard","permanent")[border_index]}_{"yes" if shadow else "no"}-shadow_x{x}_y{y}.png')
                    if not os.path.exists(destination.rsplit(os.path.sep,1)[0]):
                        os.makedirs(destination.rsplit(os.path.sep,1)[0])

                    cv2.imwrite(destination, resized_image)

# Generate the images

In [ ]:
# Get champion and skins assets
champions_and_skins = glob.glob(os.path.join("model_training","assets","champions_and_skins","*.jpg"))

# Get ward skin assets while ignoring their shadow png image
ward_skins = glob.glob(os.path.join("model_training","assets","ward_skins","*.png"))
ward_skins = list(filter(lambda ward: "shadow" not in ward, ward_skins))

# Generate training data if the lookup_table contains the ID
# If ID is missing in lookup table run "generate_lookup_table.py"
# If ID is still missing, this champion is very fresh as wikipedia doesn't know of him :)
missing_ids = []

for image in champions_and_skins:
    champion_id = image.rsplit(os.path.sep,1)[1].split('.')[0]
    if champion_id in lookup_table:
        print("Generating training data for: "+lookup_table[champion_id][0])
        generate_champion_and_skin_training_images(image)
    else:
        missing_ids.append(champion_id)
        print("Warning: Champion with ID: ["+champion_id+"] couldn't be found.")

for image in ward_skins:
    ward_id = "w" + image.rsplit("_",1)[1].split(".")[0]
    if ward_id in lookup_table:
        print("Generating training data for: "+lookup_table[ward_id][0])
        generate_ward_skin_training_images(image)
    else:
        missing_ids.append(ward_id)
        print("Warning: Ward skin with ID: ["+ward_id+"] couldn't be found. Ignored.")

if len(missing_ids) > 0:
    print("#################################################")
    print("Missing IDs (read comments in code for more info)")
    for id in missing_ids:
        print(id)


In [ ]:
train_image_paths = glob.glob( os.path.join("model_training","training_data","**","*.png") )

train_labels = []
train_images = []

for i in range(len(train_image_paths)):
    if i%1000 == 0:
        print("Progress: " + str(i//1000) + "/" + str(len(train_image_paths)//1000))

    image = train_image_paths[i]
    asset_id = image.rsplit(os.path.sep, 2)[1]
    label = list(lookup_table.keys()).index(asset_id)

    image = read_png(image)
    image = image / 255

    train_labels.append(label)
    train_images.append(image)

In [ ]:
test_image_paths = glob.glob(os.path.join("model_training","testing_data","28","**","*.png"))
test_labels = []
test_images = []


for i in range(len(test_image_paths)):
    if i%100 == 0:
        print( str(i//100) + os.path.sep + str(len(test_image_paths)//100))

    image = test_image_paths[i]
    asset_id = image.rsplit(os.path.sep, 2)[1]
    label = list(lookup_table.keys()).index(asset_id)

    image = read_png(image)
    image = image[:,:,0] / 255

    test_labels.append(label)
    test_images.append(image)

In [ ]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i*12500])
    plt.xlabel(lookup_table[list(lookup_table.keys())[train_labels[i*12500]]])
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[i*5])
    plt.xlabel(lookup_table[list(lookup_table.keys())[test_labels[i*5]]])
plt.show()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    # tf.keras.layers.Dense(len(champions)*2, activation='relu'),
    tf.keras.layers.Dense(len(lookup_table))
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit( train_images, train_labels, epochs=50)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_images)

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(lookup_table[list(lookup_table.keys())[predicted_label]],
                                100*np.max(predictions_array),
                                lookup_table[list(lookup_table.keys())[true_label]]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 32
num_cols = 14
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, i+1)
  plot_image(i, predictions[i], test_labels, test_images)
plt.tight_layout()
plt.show()

In [ ]:
model.summary()

model.save                             (os.path.join("model_training","models","model_champions_skins_wards","model.h5"))
model.save                             (os.path.join("model_training","models","model_champions_skins_wards","model"))
model.save_weights                     (os.path.join("model_training","models","model_champions_skins_wards","model_weights_checkpoint","model_weights"))
tfjs.converters.save_keras_model(model, os.path.join("model_training","models","model_champions_skins_wards","model_js"))

# Loot icon number/count recognition model

In [ ]:
number_train_labels = []
number_train_images = []

numbers_image = "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEYAAACTAQMAAAAJE/1bAAAAAXNSR0IB2cksfwAAAAlwSFlzAAALEwAACxMBAJqcGAAAAAZQTFRFAAAA////pdmf3QAAAhFJREFUeJyNzz9r20AYx/GjGKNBlDNdBA1BQ4aMJwLtmYRGAbVT+h5cktGGZHOoWrtOiDUYTDaHhPY1ZAt0qHHFNYMSz4YMNinemlZ4afDSe05/fFLkNNvDDZ/7fRHGW8Nm/ZVGkLWLTUd5YjnIKmDk4JzJL4xQvYAUcS3VjxRLQSZu6PUjjT9gwzcPVKzrCJObUfvgI90BZSNUDNzgitVCFkHmp0Jumb8Rc6krFIs0dEcoxLjrNlVMdUT/o+gZSg8UN1SeagllUyj5lEKs/YK6DJf5Iq3QUCkFymGGouVeC8Xdf6bacJVecsVNKMW7bvuRSvu5rGhoW1KmoFQjpQaKjzdaKcUl5KeknCksUC5ipT9fWcG9pOKBst71QSmTHnlIIUJpCaUlFDZfYUllEirlCQVlExSNK/m3oXICiisUR1K0lNI/rK1VQdny1EjxYqX/NVKGM+VYKHvTYagYE58rtqwwUBbyFVCMMSjXaWWgMRooi9SOt1ChvPOUjC1ZRcdhUXJLVlFlVpS1JS6CLcWg6C8qT4VyeYrYKlcWhULjIlpaH/kdxn+jPTKYFX2mc5Sr1Y6sfAOFPazsScqNt/BmpvyKlQ+RcioURgbTWPlC7T/3lT4o53MVG5Qdrowpqr5PKE1J+X0NyvdQKUbKWFZ208oaKO17yo+0cktLtVApgtIJlFGg2LFSeYxyFSu3kvIP9ENXILgWa2YAAAAASUVORK5CYII="
encoded_data = numbers_image.split(',')[1]
nparr = np.frombuffer(base64.b64decode(encoded_data), np.uint8)
img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)[:,:,0] / 255

for x in range(0, img.shape[1], 7):
    for y in range(0, img.shape[0], 7):
        crop = img[y:y+7, x:x+7]

        number_train_labels.append(x//7)
        number_train_images.append(crop)

model_number = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(7, 7)),
    tf.keras.layers.Dense(10)
])

model_number.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

number_train_labels = np.array(number_train_labels)
number_train_images = np.array(number_train_images)

model_number.fit( number_train_images, number_train_labels, epochs=5000)

model_number.summary()

model_number.save                             (os.path.join("model_training","models","model_numbers","model.h5"))
model_number.save                             (os.path.join("model_training","models","model_numbers","model"))
model_number.save_weights                     (os.path.join("model_training","models","model_numbers","model_weights_checkpoint","model_weights"))
tfjs.converters.save_keras_model(model_number, os.path.join("model_training","models","model_numbers","model_js"))

# Shard/Permanent model

In [ ]:
shard_permanent_image_paths = glob.glob( os.path.join("model_training","training_data","**","*.png") )

shard_permanent_train_labels = []
shard_permanent_train_images = []

for i in range(len(shard_permanent_image_paths)):
    if i%1000 == 0:
        print("Progress: " + str(i//1000) + "/" + str(len(shard_permanent_image_paths)//1000))

    image = shard_permanent_image_paths[i]
    label = 0 if "shard" in image else 1

    image = read_png(image) / 255

    shard_permanent_train_labels.append(label)
    shard_permanent_train_images.append(image)

shard_permanent_train_images = np.array(shard_permanent_train_images)
shard_permanent_train_labels = np.array(shard_permanent_train_labels)

model_shard_permanent = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(2)
])

model_shard_permanent.compile(optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
    
model_shard_permanent.fit( shard_permanent_train_images, shard_permanent_train_labels, epochs=10)

model_shard_permanent.summary()

model_shard_permanent.save                             (os.path.join("model_training","models","model_shard_permanent","model.h5"))
model_shard_permanent.save                             (os.path.join("model_training","models","model_shard_permanent","model"))
model_shard_permanent.save_weights                     (os.path.join("model_training","models","model_shard_permanent","model_weights_checkpoint","model_weights"))
tfjs.converters.save_keras_model(model_shard_permanent, os.path.join("model_training","models","model_shard_permanent","model_js"))